In [669]:
import pandas as pd
from fuzzywuzzy import process

In this Project we will try finding a model good enough to predict the Footbal players value uo to 2 years forward because Short-term predictions are generally more accurate as they rely on recent performance data, current form, and market trends. Factors like a player's recent injuries, transfer rumors, and current performance are more predictable over a short period.
These predictions are useful for clubs and agents making immediate decisions about contracts and transfers.

# **Loading and Trasforming the Data frame**

In [670]:
#load the data of transfers for each leage in each year from 2017-2021
# 2021
df_laliga_2021 = pd.read_csv(r'2021\laliga_2021.csv')
df_bundesliga_2021 = pd.read_csv(r'2021\bundesliga_2021.csv')
df_liga1_2021 = pd.read_csv(r'2021\ligue-1_2021.csv')
df_premier_2021 = pd.read_csv(r'2021\premier-league_2021.csv')
df_seria_2021 = pd.read_csv(r'2021\serie-a_2021.csv')

# 2020
df_laliga_2020 = pd.read_csv(r'2020\laliga_2020.csv')
df_bundesliga_2020 = pd.read_csv(r'2020\bundesliga_2020.csv')
df_liga1_2020 = pd.read_csv(r'2020\ligue-1_2020.csv')
df_premier_2020 = pd.read_csv(r'2020\premier-league_2020.csv')
df_seria_2020 = pd.read_csv(r'2020\serie-a_2020.csv')

# 2019
df_laliga_2019 = pd.read_csv(r'2019\laliga_2019.csv')
df_bundesliga_2019 = pd.read_csv(r'2019\bundesliga_2019.csv')
df_liga1_2019 = pd.read_csv(r'2019\ligue-1_2019.csv')
df_premier_2019 = pd.read_csv(r'2019\premier-league_2019.csv')
df_seria_2019 = pd.read_csv(r'2019\serie-a_2019.csv')

# 2018
df_laliga_2018 = pd.read_csv(r'2018\laliga_2018.csv')
df_bundesliga_2018 = pd.read_csv(r'2018\bundesliga_2018.csv')
df_liga1_2018 = pd.read_csv(r'2018\ligue-1_2018.csv')
df_premier_2018 = pd.read_csv(r'2018\premier-league_2018.csv')
df_seria_2018 = pd.read_csv(r'2018\serie-a_2018.csv')

# 2017
df_laliga_2017 = pd.read_csv(r'2017\laliga_2017.csv')
df_bundesliga_2017 = pd.read_csv(r'2017\bundesliga_2017.csv')
df_liga1_2017 = pd.read_csv(r'2017\ligue-1_2017.csv')
df_premier_2017 = pd.read_csv(r'2017\premier-league_2017.csv')
df_seria_2017 = pd.read_csv(r'2017\serie-a_2017.csv')

# List of all DataFrames
league_year_dfs = [
    df_laliga_2021, df_bundesliga_2021, df_liga1_2021, df_premier_2021, df_seria_2021,
    df_laliga_2020, df_bundesliga_2020, df_liga1_2020, df_premier_2020, df_seria_2020,
    df_laliga_2019, df_bundesliga_2019, df_liga1_2019, df_premier_2019, df_seria_2019,
    df_laliga_2018, df_bundesliga_2018, df_liga1_2018, df_premier_2018, df_seria_2018,
    df_laliga_2017, df_bundesliga_2017, df_liga1_2017, df_premier_2017, df_seria_2017
]
df_2021 = pd.concat([df_laliga_2021, df_bundesliga_2021, df_liga1_2021, df_premier_2021, df_seria_2021],ignore_index=True)
df_2020 = pd.concat([ df_laliga_2020, df_bundesliga_2020, df_liga1_2020, df_premier_2020, df_seria_2020],ignore_index=True)
df_2019 = pd.concat([df_laliga_2019, df_bundesliga_2019, df_liga1_2019, df_premier_2019, df_seria_2019],ignore_index=True)
df_2018 = pd.concat([df_laliga_2018, df_bundesliga_2018, df_liga1_2018, df_premier_2018, df_seria_2018],ignore_index=True)
df_2017 = pd.concat([df_laliga_2017, df_bundesliga_2017, df_liga1_2017, df_premier_2017, df_seria_2017],ignore_index=True)
df_transfers_list = [df_2021, df_2020, df_2019, df_2018, df_2017]
# Combine all yearly DataFrames into one DataFrame for transfers
combined_years_top5_leagues = pd.concat(league_year_dfs, ignore_index=True)
combined_years_top5_leagues




,club,name,age,nationality,position,short_pos,market_value,dealing_club,dealing_country,fee,movement,window,league,season,is_loan,loan_status
0,FC Barcelona,Rodrigo de Paul,27,Argentina,Central Midfield,CM,40000000.00,Udinese Calcio,Italy,35000000.00,in,summer,Laliga,2021,False,NaN
1,FC Barcelona,Matheus Cunha,22,Brazil,Centre-Forward,CF,30000000.00,Hertha BSC,Germany,26000000.00,in,summer,Laliga,2021,False,NaN
2,FC Barcelona,Antoine Griezmann,30,France,Second Striker,SS,50000000.00,Barcelona,Spain,10000000.00,in,summer,Laliga,2021,True,loan with fee
3,FC Barcelona,Marcos Paulo,20,Portugal,Left Winger,LW,9000000.00,Fluminense,Brazil,0.00,in,summer,Laliga,2021,False,NaN
4,FC Barcelona,Benjamin Lecomte,30,France,Goalkeeper,GK,9000000.00,Monaco,France,0.00,in,summer,Laliga,2021,True,free loan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14960,Udinese Calcio,Riad Bajić,23,Bosnia-Herzegovina,Centre-Forward,CF,4500000.00,Basaksehir,Turkey,0.00,out,winter,Serie A,2017,True,free loan
14961,Udinese Calcio,Ewandro Costa,21,Brazil,Right Winger,RW,1000000.00,Estoril Praia,Portugal,0.00,out,winter,Serie A,2017,True,free loan
14962,Udinese Calcio,Xabier Santos,20,Chile,Left Winger,LW,25000.00,Ñublense,Chile,0.00,out,winter,Serie A,2017,True,free loan
14963,Udinese Calcio,Nabil Jaadi,21,Morocco,Right Winger,RW,100000.00,Asteras Tripoli,Greece,0.00,out,winter,Serie A,2017,True,free loan


In [671]:
## Printing number of records in the transformed dataFrame
print('Number of records in the dataFrame:', len(combined_years_top5_leagues))
print('The columns of the dataFrame:', list(combined_years_top5_leagues.columns))
print('The types of the transformed dataFrames variables:\n', combined_years_top5_leagues.dtypes)

Number of records in the dataFrame: 14965
The columns of the dataFrame: ['club', 'name', 'age', 'nationality', 'position', 'short_pos', 'market_value', 'dealing_club', 'dealing_country', 'fee', 'movement', 'window', 'league', 'season', 'is_loan', 'loan_status']
The types of the transformed dataFrames variables:
 club                object
name                object
age                  int64
nationality         object
position            object
short_pos           object
market_value       float64
dealing_club        object
dealing_country     object
fee                float64
movement            object
window              object
league              object
season               int64
is_loan               bool
loan_status         object
dtype: object



The data we are working with contains the real transfer values of players from the top 5 football leagues: the English Premier League, La Liga (Spain), Bundesliga (Germany), Serie A (Italy), and Ligue 1 (France). This dataset will allow us to analyze player transfers, compare market trends across different leagues, and potentially identify patterns in player valuations.
This data includes years 2017-2021 only, we are going to add more years in the next steps. This data was taken from transfermarkt: A popular website that provides detailed information on player transfers, including transfer fees, player stats, and historical data for many leagues, including the top 5 European leagues.


In [672]:
# Load the player transfers data from the top 5 leagues across the last 12 years
df_transfers_top5 = pd.read_csv(r'players_transfers__top5_leagues.csv')

# Display the dataframe (for checking if the data loaded correctly)
df_transfers_top5


,Unnamed: 0,name,full_name,age,height,nationality,place_of_birth,price,max_price,position,shirt_nr,foot,club,contract_expires,joined_club,player_agent,outfitter,league
0,1144,Enzo Barrenechea,Enzo Alan Tomás Barrenechea,21,1.86,Argentina,Villa María,0.90,0.90,midfield - Defensive Midfield,45,right,Juventus,6/30/2026,4/1/2023,SP,NaN,SerieA
1,1328,Marvin Zeegelaar,Marvin Romeo Kwasie Zeegelaar,32,1.86,Netherlands Suriname,Amsterdam,0.40,4.00,Defender - Left-Back,8,left,Udinese Calcio,6/30/2023,3/7/2023,NaN,NaN,SerieA
2,2374,Cheick Keita,NaN,20,1.85,France Mali,Champigny-sur-Marne,1.50,1.50,Defender - Centre-Back,43,right,Stade Reims,6/30/2026,3/1/2023,Nessim Zeggaie,NaN,Ligue1
3,1555,Marios Oikonomou,Μάριος Οικονόμου,30,1.89,Greece,Ioannina,0.30,2.50,Defender - Centre-Back,5,right,Sampdoria,6/30/2023,2/17/2023,SPORTS CONNECTION,NaN,SerieA
4,2085,Pape Cheikh,Pape Cheikh Diop Gueye,25,1.80,Senegal Spain,Guédiawaye,1.50,7.00,midfield - Central Midfield,12,right,Elche CF,6/30/2023,2/14/2023,4MB,Nike,LaLiga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1764,Iker Muniain,Iker Muniain Goñi,30,1.69,Spain,Pamplona,10.00,22.00,midfield - Attacking Midfield,10,right,Athletic,6/30/2024,7/1/2009,Jesus Medina Martin,NaN,LaLiga
2608,1828,Hugo Mallo,Hugo Mallo Novegil,31,1.74,Spain,Marín,3.50,15.00,Defender - Right-Back,2,right,Celta de Vigo,6/30/2023,7/1/2009,#LEADERS,NaN,LaLiga
2609,361,Seamus Coleman,Séamus Coleman,34,1.77,Ireland,Donegal,1.20,19.00,Defender - Right-Back,23,right,Everton,6/30/2023,1/1/2009,NaN,Nike,EPL
2610,1218,Stefan Radu,Ștefan Daniel Radu,36,1.83,Romania,Bukarest,0.40,10.00,Defender - Left-Back,26,left,Lazio,6/30/2023,7/1/2008,Matteo Materazzi,adidas,SerieA


In [673]:
## Printing number of records in the transformed dataFrame
print('Number of records in the dataFrame:', len(df_transfers_top5))
print('The columns of the dataFrame:', list(df_transfers_top5.columns))
print('The types of the transformed dataFrames variables:\n', df_transfers_top5.dtypes)

Number of records in the dataFrame: 2612
The columns of the dataFrame: ['Unnamed: 0', 'name', 'full_name', 'age', 'height', 'nationality', 'place_of_birth', 'price', 'max_price', 'position', 'shirt_nr', 'foot', 'club', 'contract_expires', 'joined_club', 'player_agent', 'outfitter', 'league']
The types of the transformed dataFrames variables:
 Unnamed: 0            int64
name                 object
full_name            object
age                   int64
height              float64
nationality          object
place_of_birth       object
price               float64
max_price           float64
position             object
shirt_nr              int64
foot                 object
club                 object
contract_expires     object
joined_club          object
player_agent         object
outfitter            object
league               object
dtype: object


Top 5 Football Leagues Playprofiles Dataset (2022/23 Season). This dataset includes real and essential details such as player names, clubs, positions, heights, preferred foot, agents, outfitters, the price shows the current price in millions and max price is The maximum price the player ever had in millions(dollars). The data was found in kaggle.

In [674]:

# Rename the column 'price' to 'fee'
df_transfers_top5.rename(columns={'price': 'fee'}, inplace=True)

# Convert the 'joined_club' column to datetime format
df_transfers_top5['joined_club'] = pd.to_datetime(df_transfers_top5['joined_club'])

# Filter the dataframe to include only transfers after January 1, 2022
df_transfers_top5 = df_transfers_top5[df_transfers_top5['joined_club'] > '2022-01-01']

# Extract the year from the 'joined_club' date and create a new column 'year'
df_transfers_top5['year'] = df_transfers_top5['joined_club'].dt.year

# Select relevant columns for analysis
df_transfers_top5 = df_transfers_top5[['name', 'age', 'height', 'nationality', 'fee', 'max_price', 'position', 'foot', 'year', 'league']]

# Filter out transfers from leagues other than the top 5
df_transfers_top5 = df_transfers_top5[df_transfers_top5['league'] != 'Other']

# Extract the transfer data for the years 2022 and 2023
df_transfers_top5_2023 = df_transfers_top5[df_transfers_top5['year'] == 2023]
df_transfers_top5_2022 = df_transfers_top5[df_transfers_top5['year'] == 2022]

# Display the columns of the 2021 dataframe for checking and help us understand what data we want to keep
print(df_2021.columns.tolist())


['club', 'name', 'age', 'nationality', 'position', 'short_pos', 'market_value', 'dealing_club', 'dealing_country', 'fee', 'movement', 'window', 'league', 'season', 'is_loan', 'loan_status']


C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\2153331471.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transfers_top5['year'] = df_transfers_top5['joined_club'].dt.year


In [675]:

#filter the relevant columns
df_transfers_top5_2023 = df_transfers_top5_2023[['name', 'age', 'fee', 'position', 'league']]
df_transfers_top5_2022 = df_transfers_top5_2022[['name', 'age', 'fee', 'position', 'league']]


In [676]:
#Because we found out that not all the names are at the same format across the data sets we want to add column of first name and last name combined
# so it will be the same format across the data sets

# This method extracts the last name from the 
def extract_last_name(full_name):
    name_parts = full_name.split(' ')
    if len(name_parts) > 1:
        return ' '.join(name_parts[1:])
    return full_name

# This method extracte the first name
def extract_first_name(full_name):
    name_parts = full_name.split(' ')
    return ' '.join(name_parts[:1])


#EXAMPLE
df = df_laliga_2021.copy()
# combine the first and last name to create full name
df['last_name'] = df['name'].apply(extract_last_name)
df['first_name'] = df['name'].apply(extract_first_name)
df['full_name'] = df['first_name'] + ' ' + df['last_name']
df.head()

,club,name,age,nationality,position,short_pos,market_value,dealing_club,dealing_country,fee,movement,window,league,season,is_loan,loan_status,last_name,first_name,full_name
0,FC Barcelona,Rodrigo de Paul,27,Argentina,Central Midfield,CM,40000000.00,Udinese Calcio,Italy,35000000.00,in,summer,Laliga,2021,False,NaN,de Paul,Rodrigo,Rodrigo de Paul
1,FC Barcelona,Matheus Cunha,22,Brazil,Centre-Forward,CF,30000000.00,Hertha BSC,Germany,26000000.00,in,summer,Laliga,2021,False,NaN,Cunha,Matheus,Matheus Cunha
2,FC Barcelona,Antoine Griezmann,30,France,Second Striker,SS,50000000.00,Barcelona,Spain,10000000.00,in,summer,Laliga,2021,True,loan with fee,Griezmann,Antoine,Antoine Griezmann
3,FC Barcelona,Marcos Paulo,20,Portugal,Left Winger,LW,9000000.00,Fluminense,Brazil,0.00,in,summer,Laliga,2021,False,NaN,Paulo,Marcos,Marcos Paulo
4,FC Barcelona,Benjamin Lecomte,30,France,Goalkeeper,GK,9000000.00,Monaco,France,0.00,in,summer,Laliga,2021,True,free loan,Lecomte,Benjamin,Benjamin Lecomte


In [677]:
df_player_transfer = pd.read_csv('players_transfers__top5_leagues.csv')


In [678]:
# Function to preprocess transfer data
def preprocess_transfers(df):
    # Filter out transfers with zero or missing fee
    df = df[(df['fee'] != 0) & (df['fee'].notna())]

    # Find and print duplicate records
    duplicates = df[df.duplicated()]
    print("Duplicate records found:", len(duplicates))

    # Select relevant columns
    df = df[['name', 'age', 'fee', 'position', 'league']]
    return df

# Initialize a variable to count the total number of transfers
num_transfers = 0

# Process each transfer dataframe in the list
for i in range(len(df_transfers_list)):
    print("Before processing:", len(df_transfers_list[i]))
    df_transfers_list[i] = preprocess_transfers(df_transfers_list[i])
    num_transfers += len(df_transfers_list[i])
    print("After processing:", len(df_transfers_list[i]))

# Print the total number of transfers
print("Total transfers:", num_transfers)

# Print the number of dataframes in the list
print("Number of dataframes in list:", len(df_transfers_list))

# Combine all processed transfer dataframes into a single list
df_transfers_all = df_transfers_list + [df_transfers_top5_2023, df_transfers_top5_2022] 

# Print the total number of dataframes in the combined list
print("Number of dataframes in combined list:", len(df_transfers_all))


Before processing: 2211
Duplicate records found: 0
After processing: 617
Before processing: 2784
Duplicate records found: 0
After processing: 783
Before processing: 3384
Duplicate records found: 0
After processing: 1211
Before processing: 3276
Duplicate records found: 0
After processing: 1219
Before processing: 3310
Duplicate records found: 0
After processing: 1194
Total transfers: 5024
Number of dataframes in list: 5
Number of dataframes in combined list: 7


In [697]:
# checking that all dataframs have same columns

for df in df_transfers_all:
  # Check if "Messi" is in the DataFrame
  df = df[df['name'].str.contains('Messi')]

  # Display the entries related to Messi
  if not df.empty:
      print("Messi is in the dataset.")
  else:
      print("Messi is not in the dataset.")


  print (df.columns.tolist())

Messi is in the dataset.
['name', 'age', 'fee', 'position', 'league']
Messi is not in the dataset.
['name', 'age', 'fee', 'position', 'league']
Messi is not in the dataset.
['name', 'age', 'fee', 'position', 'league']
Messi is not in the dataset.
['name', 'age', 'fee', 'position', 'league']
Messi is not in the dataset.
['name', 'age', 'fee', 'position', 'league']
Messi is not in the dataset.
['name', 'age', 'fee', 'position', 'league']
Messi is in the dataset.
['name', 'age', 'fee', 'position', 'league']


In [680]:
pd.set_option('display.float_format', '{:.2f}'.format)
df.describe()


,age,fee
count,755.00,753.00
mean,24.80,9.95
std,4.33,14.13
min,17.00,0.03
25%,21.00,2.00
50%,24.00,5.00
75%,28.00,12.00
max,41.00,170.00


### **Loading Transforming and Feature Extraction from the FIFA Fata frame**
- need to decide which features to take from FIFA

In [681]:

def pre_process_fifa(df):
    # Extract first name and last name (ignoring middle names)
    df['last_name'] = df['long_name'].apply(extract_last_name)
    
    # Replace 'Cuccittini' with 'Messi' because messi name in the data set is 
    df['last_name'] = df['last_name'].replace('Cuccittini', 'Messi')
    
    df['first_name'] = df['long_name'].apply(extract_first_name)
    
    # Combine first name and last name
    df['name'] = df['first_name'] + ' ' + df['last_name']
    
    standard_columns = {
        'nationality': 'nationality_name',
        'club': 'club_name',
        'defending_marking': 'defending_marking_awareness',
        'gk_diving': 'goalkeeping_diving',
        'gk_handling': 'goalkeeping_handling',
        'gk_kicking': 'goalkeeping_kicking',
        'gk_reflexes': 'goalkeeping_reflexes',
        'gk_speed': 'goalkeeping_speed',
        'gk_positioning': 'goalkeeping_positioning',
        'team_position': 'club_position',
        'team_jersey_number': 'club_jersey_number',
        'loaned_from': 'club_loaned_from',
        'joined': 'club_joined',
        'contract_valid_until': 'club_contract_valid_until',
        'age': 'fifa_age'
    }
    
    df.rename(columns=standard_columns, inplace=True)
    
    columns_to_drop = ['player_url', 'dob', 'club_jersey_number', 'club_loaned_from', 'league_id', 'fifa_age',
                       'body_type', 'nation_jersey_number', 'release_clause_eur', 'real_face', 'last_name', 'first_name',
                       'club_contract_valid_until', 'wage_eur', 'club_logo_url', 'club_flag_url', 'club_team_id', 'nationality_id', 'value_eur',
                       'player_id', 'fifa_update', 'update_as_of', 'short_name', 'long_name', 'club_team_id', 'club_joined_date', 'nation_team_id',
                       'nation_logo_url', 'nation_flag_url', 'league_name', 'league_rank', 'league_level', 'sofifa_id', 'player_face_url']
    
    columns_existing_to_drop = [col for col in columns_to_drop if col in df.columns]
    df = df.drop(columns=columns_existing_to_drop)
    
    return df


In [696]:
# Load the FIFA 24 dataset
df_24 = pd.read_csv('Fifa_24.csv')

# Filter the dataframe to include only entries where the 'fifa_version' is 17 or higher
df_24 = df_24[df_24['fifa_version'] >= 17]

# Display the first few rows of the filtered dataframe
df_24.head()
df_24=pre_process_fifa (df_24)

# Check if "Messi" is in the DataFrame
messi_df = df_24[df_24['name'].str.contains('Messi')]

# Display the entries related to Messi
if not messi_df.empty:
    print("Messi is in the dataset.")
else:
    print("Messi is not in the dataset.")

C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\2160974083.py:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_24 = pd.read_csv('Fifa_24.csv')


Messi is in the dataset.


The datasets provided include the players data for the Career Mode from FIFA 15 to EA Sports FC 24. The data allows multiple comparisons for the same players across the last 10 versions of the videogame.The data was taken from kaggle and  has been scraped from the publicly available website sofifa.com. The website is particularly well-known for its extensive database of player attributes, ratings, and statistics as they appear in the FIFA games.

In [683]:

# Initialize an empty DataFrame with the same columns as df_24
full_joined_df = pd.DataFrame(columns=df_24.columns.tolist())
full_joined_df

,fifa_version,player_positions,overall,potential,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,nationality_name,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,name


In [684]:
import pandas as pd
from rapidfuzz import process, fuzz

# Function to perform fuzzy matching
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    df_1: the left table to join
    df_2: the right table to join
    key1: key column of the left table
    key2: key column of the right table
    threshold: how close the matches should be to return a match, based on Levenshtein distance
    limit: the amount of matches that will get returned, these are sorted high to low
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extractOne(x, s, scorer=fuzz.ratio))
    
    df_1['match_score'] = m.apply(lambda x: x[1] if x is not None else None)
    df_1[key2] = m.apply(lambda x: x[0] if x is not None else None)
    
    return df_1[df_1['match_score'] >= threshold].drop(columns=['match_score'])

# Create an empty DataFrame with the same columns as df_24
full_joined_df = pd.DataFrame(columns=df_24.columns.tolist())

for i in range(len(df_transfers_all)):
    df_fifa_this_year = df_24[df_24['fifa_version'] == (24 - i)]
    
    # Use fuzzy matching to merge FIFA data with transfer data
    fifa_with_transfer = fuzzy_merge(df_fifa_this_year, df_transfers_all[i], 'name', 'name')
    
    # Drop duplicates to keep the most relevant entry
    fifa_with_transfer = fifa_with_transfer.drop_duplicates(subset=['name'])
    
    # Concatenate with the accumulated DataFrame
    full_joined_df = pd.concat([full_joined_df, fifa_with_transfer], ignore_index=True)
    
    print(f"Iteration {i}: Unique FIFA versions - {full_joined_df['fifa_version'].unique()}")

# Display the final DataFrame
print(full_joined_df.columns.tolist())
display(full_joined_df)
print(len(full_joined_df))

# Save to CSV
full_joined_df.to_csv('transfer_fifa_player_data_messi.csv', index=False)


C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\727092920.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['match_score'] = m.apply(lambda x: x[1] if x is not None else None)
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\727092920.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[key2] = m.apply(lambda x: x[0] if x is not None else None)
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\727092920.py:36: FutureWarning: The behavior of DataFrame concatenation 

Iteration 0: Unique FIFA versions - [24.]


C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\727092920.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['match_score'] = m.apply(lambda x: x[1] if x is not None else None)
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\727092920.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[key2] = m.apply(lambda x: x[0] if x is not None else None)


Iteration 1: Unique FIFA versions - [24. 23.]


C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\727092920.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['match_score'] = m.apply(lambda x: x[1] if x is not None else None)
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\727092920.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[key2] = m.apply(lambda x: x[0] if x is not None else None)


...

C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\727092920.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['match_score'] = m.apply(lambda x: x[1] if x is not None else None)
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\727092920.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[key2] = m.apply(lambda x: x[0] if x is not None else None)


Iteration 5: Unique FIFA versions - [24. 23. 22. 21. 20. 19.]
Iteration 6: Unique FIFA versions - [24. 23. 22. 21. 20. 19. 18.]
['fifa_version', 'player_positions', 'overall', 'potential', 'height_cm', 'weight_kg', 'club_name', 'club_position', 'club_contract_valid_until_year', 'nationality_name', 'nation_position', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_position

C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\727092920.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['match_score'] = m.apply(lambda x: x[1] if x is not None else None)
C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\727092920.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[key2] = m.apply(lambda x: x[0] if x is not None else None)


,fifa_version,player_positions,overall,potential,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,nationality_name,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,name
0,24.00,"ST, CF",88,88,176,73,Atlético Madrid,RS,2026.00,France,...,74+3,74+3,75+3,73+3,68+3,68+3,68+3,73+3,21+3,Antoine Griezmann
1,24.00,"CAM, CM",87,90,178,68,Arsenal,RCM,2026.00,Norway,...,75+3,75+3,75+3,71+3,64+3,64+3,64+3,71+3,20+3,Martin Ødegaard
2,24.00,GK,87,90,195,88,Borussia Dortmund,GK,2026.00,Switzerland,...,33+3,33+3,29+3,28+3,30+3,30+3,30+3,28+3,85+3,Gregor Kobel
3,24.00,GK,87,89,191,89,Milan,GK,2026.00,France,...,43+3,43+3,37+3,35+3,35+3,35+3,35+3,35+3,85+3,Mike Maignan
4,24.00,"CDM, CM",86,89,181,79,Newcastle United,RCM,2028.00,Italy,...,84+3,84+3,83+3,83+3,82+3,82+3,82+3,83+3,17+3,Sandro Tonali
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876,18.00,GK,57,70,196,90,Heidenheim,RES,2019.00,Germany,...,24+1,24+1,20+1,20+1,23+1,23+1,23+1,20+1,56+1,Matthias Köbbing
1877,18.00,ST,56,74,188,78,FC Carl Zeiss Jena,RS,2019.00,Germany,...,34+1,34+1,35+1,34+1,34+1,34+1,34+1,34+1,14+1,Florian Dietz
1878,18.00,"CF, CAM, ST",56,73,186,79,LASK Linz,SUB,2018.00,Kosovo,...,38+1,38+1,40+1,38+1,33+1,33+1,33+1,38+1,13+1,Mergim Berisha
1879,18.00,"CM, CDM",55,66,182,75,Young Boys,LCM,2020.00,Switzerland,...,55+1,55+1,55+1,54+1,52+1,52+1,52+1,54+1,15+1,Michel Aebischer


1881


In [685]:
full_joined_df = pd.DataFrame(columns = df_24.columns.tolist())
for i in range (len(df_transfers_all)):
    df_fifa_this_year = df_24[df_24['fifa_version'] ==(24 - i)]
    fifa_with_transfer = pd.merge(df_fifa_this_year, df_transfers_all[i], on='name', how='inner')
    fifa_with_transfer = fifa_with_transfer.drop_duplicates(subset=['name'])
    dfs_to_merge = [full_joined_df, fifa_with_transfer]
    full_joined_df = pd.concat(dfs_to_merge, ignore_index=True)
    print("iteration", i, full_joined_df['fifa_version'].unique())

print((full_joined_df.columns.tolist()))
display(full_joined_df)
print(len(full_joined_df))
full_joined_df.to_csv('transfer_fifa_player_data.csv', index=False)





C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\2223143451.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_joined_df = pd.concat(dfs_to_merge, ignore_index=True)


iteration 0 [24.]
iteration 1 [24. 23.]
iteration 2 [24. 23. 22.]
iteration 3 [24. 23. 22. 21.]
iteration 4 [24. 23. 22. 21. 20.]
iteration 5 [24. 23. 22. 21. 20. 19.]
iteration 6 [24. 23. 22. 21. 20. 19. 18.]
['fifa_version', 'player_positions', 'overall', 'potential', 'height_cm', 'weight_kg', 'club_name', 'club_position', 'club_contract_valid_until_year', 'nationality_name', 'nation_position', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_l

,fifa_version,player_positions,overall,potential,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,nationality_name,...,lcb,cb,rcb,rb,gk,name,age,fee,position,league
0,24.00,"ST, CF",88,88,176,73,Atlético Madrid,RS,2026.00,France,...,68+3,68+3,68+3,73+3,21+3,Antoine Griezmann,30.00,10000000.00,Second Striker,Laliga
1,24.00,"CAM, CM",87,90,178,68,Arsenal,RCM,2026.00,Norway,...,64+3,64+3,64+3,71+3,20+3,Martin Ødegaard,22.00,35000000.00,Attacking Midfield,Laliga
2,24.00,GK,87,90,195,88,Borussia Dortmund,GK,2026.00,Switzerland,...,30+3,30+3,30+3,28+3,85+3,Gregor Kobel,23.00,15000000.00,Goalkeeper,1 Bundesliga
3,24.00,GK,87,89,191,89,Milan,GK,2026.00,France,...,35+3,35+3,35+3,35+3,85+3,Mike Maignan,25.00,14300000.00,Goalkeeper,Ligue 1
4,24.00,"CDM, CM",86,89,181,79,Newcastle United,RCM,2028.00,Italy,...,82+3,82+3,82+3,83+3,17+3,Sandro Tonali,21.00,6900000.00,Defensive Midfield,Serie A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,18.00,"CB, LB",57,72,185,75,Lens,RES,2020.00,France,...,56+1,56+1,56+1,55+1,15+1,Enzo Ebosse,24.00,4.50,Defender - Centre-Back,SerieA
1691,18.00,GK,57,70,196,90,Heidenheim,RES,2019.00,Germany,...,23+1,23+1,23+1,20+1,56+1,Matthias Köbbing,25.00,0.03,Goalkeeper,Bundesliga
1692,18.00,ST,56,74,188,78,FC Carl Zeiss Jena,RS,2019.00,Germany,...,34+1,34+1,34+1,34+1,14+1,Florian Dietz,24.00,1.00,Attack - Centre-Forward,Bundesliga
1693,18.00,"CM, CDM",55,66,182,75,Young Boys,LCM,2020.00,Switzerland,...,52+1,52+1,52+1,54+1,15+1,Michel Aebischer,26.00,3.50,midfield - Central Midfield,SerieA


1695


[דברים לבדוק
נרמול שנים
נרמול שנת סיום חוזה לפי אורך חזה
למחוק ציון לפי כל עמדה
לעשות one hot code לכל עמדה ולשמור מחיר עמדה](https:// [link text](https://))

change positions col to 
1 - gk
2- defence
3- midfield
4- attack
#TODO so we can abandon the player_position and club_position columns

In [686]:
# for each player name add also with join his fifa data from last year 

In [687]:
full_joined_df['position'].unique()

array(['Second Striker', 'Attacking Midfield', 'Goalkeeper',
       'Defensive Midfield', 'Centre-Back', 'Left Winger',
       'Centre-Forward', 'Central Midfield', 'Right Winger', 'Right-Back',
       'Right Midfield', 'Left-Back', 'Left Midfield',
       'Attack - Centre-Forward', 'midfield - Central Midfield',
       'Defender - Centre-Back', 'Defender - Left-Back',
       'Attack - Right Winger', 'Attack - Second Striker',
       'Attack - Left Winger', 'midfield - Attacking Midfield',
       'Defender - Right-Back', 'midfield - Right Midfield',
       'midfield - Defensive Midfield', 'midfield - Left Midfield'],
      dtype=object)

In [688]:
mapping ={
'Defender - Right-Back': 2,
'Goalkeeper': 1,
'Attack - Centre-Forward': 4,
'midfield - Defensive Midfield': 3,
'Attack - Left Winger': 4,
'Attack - Right Winger': 4,
'Attack - Second Striker': 4,
'midfield - Central Midfield':3,
'Defender - Centre-Back': 2,
'midfield - Attacking Midfield': 3,
'Defender - Left-Back': 2,
'midfield - Right Midfield': 3,
'midfield - Left Midfield': 3,
'Centre-Forward': 4,
'Left Winger': 4,
'Centre-Back': 2,
'Right-Back': 2,
'Second Striker': 4,
'Central Midfield': 3,
'Attacking Midfield': 3,
'Left Midfield': 3,
'Defensive Midfield': 2,
'Right Winger': 4,
'Right Midfield': 3,
 'Left-Back': 2

}

full_joined_df['position'].head()

#MAPPING THE VALUE IN THE COLUMN OF POSITION
full_joined_df['position'] = full_joined_df['position'].map(mapping)
full_joined_df['position'].unique()

array([4, 3, 1, 2], dtype=int64)

In [689]:
pos_list = ['','GK', 'DEF', 'MID', 'ATT']
for i in range(1,5):
    print(f"number of players in position {pos_list[i]}", len(full_joined_df[full_joined_df['position'] == i]))

number of players in position GK 141
number of players in position DEF 647
number of players in position MID 384
number of players in position ATT 523


player_traits and player_tags represent special skills
we will count them for each player

In [690]:
# player tags represent the player skills
full_joined_df['player_tags'].unique()
full_joined_df['player_traits'].unique()

array(['Finesse Shot +, Chip Shot, Pinged Pass, Technical, Flair, Trivela, Acrobatic',
       'Pinged Pass +, Incisive Pass, Whipped Cross, Technical, Flair, First Touch, Trivela',
       'Far Throw, Footwork', 'Far Throw',
       'Long Ball Pass, Slide Tackle, Relentless', 'Anticipate +, Aerial',
       'Press Proven +, Finesse Shot, Flair, First Touch', 'Block', nan,
       'Finesse Shot, Flair',
       'Technical, Flair, First Touch, Trickster, Quick Step, Trivela',
       'Acrobatic +, Power Header, Trivela, Aerial',
       'Power Shot, Anticipate, Bruiser, Relentless, Trivela',
       'Technical, Flair, Trivela', 'Block, Bruiser, Aerial',
       'Flair, Relentless, Trivela',
       'Finesse Shot, Whipped Cross, Technical, Trivela',
       'Long Ball Pass, Bruiser, Long Throw, Aerial',
       'Pinged Pass, Long Ball Pass, Relentless',
       'Pinged Pass, Slide Tackle, Technical, Flair, Trivela', 'Jockey',
       'Power Shot, Intercept, Trivela', 'Slide Tackle',
       'Jockey, Int

In [691]:
#if the player didnt have skill is appear as nan so then we will define it as 0
#the skills separate by , so we will split them and then count
full_joined_df['player_tags'] = full_joined_df['player_tags'].apply(lambda x: len(set(x.split(','))) if isinstance(x, str) else 0)
full_joined_df['player_traits'] = full_joined_df['player_traits'].apply(lambda x: len(set(x.split(','))) if isinstance(x, str) else 0)



In [692]:
full_joined_df['player_tags'].unique()
full_joined_df['player_traits'].unique()

array([7, 2, 1, 3, 4, 0, 6, 5], dtype=int64)

## ADDING MORE 5 LEAGUES(OPTIONAL)

In [693]:
# 2021
df_holland = pd.read_csv('2021\eredivisie_2021.csv')
df_belgium = pd.read_csv('2021\jupiler-pro-league_2021.csv')
df_russia = pd.read_csv('2021\premier-liga_2021.csv')
df_portugal = pd.read_csv('2021\primeira-liga_2021.csv')
df_scotland = pd.read_csv('2021\scottish-premiership_2021.csv')

 # 2020
df_holland_2020 = pd.read_csv('2020\eredivisie_2020.csv')
df_belgium_2020 = pd.read_csv('2020\jupiler-pro-league_2020.csv')
df_russia_2020 = pd.read_csv('2020\premier-liga_2020.csv')
df_portugal_2020 = pd.read_csv('2020\primeira-liga_2020.csv')
df_scotland_2020 = pd.read_csv('2020\scottish-premiership_2020.csv')

# 2019
df_holland_2019 = pd.read_csv('2019\eredivisie_2019.csv')
df_belgium_2019 = pd.read_csv('2019\jupiler-pro-league_2019.csv')
df_russia_2019 = pd.read_csv('2019\premier-liga_2019.csv')
df_portugal_2019 = pd.read_csv('2019\primeira-liga_2019.csv')
df_scotland_2019 = pd.read_csv('2019\scottish-premiership_2019.csv')

# 2018
df_holland_2018 = pd.read_csv('2018\eredivisie_2018.csv')
df_belgium_2018 = pd.read_csv('2018\jupiler-pro-league_2018.csv')
df_russia_2018 = pd.read_csv('2018\premier-liga_2018.csv')
df_portugal_2018 = pd.read_csv('2018\primeira-liga_2018.csv')
df_scotland_2018 = pd.read_csv('2018\scottish-premiership_2018.csv')

# 2017
df_holland_2017 = pd.read_csv('2017\eredivisie_2017.csv')
df_belgium_2017 = pd.read_csv('2017\jupiler-pro-league_2017.csv')
df_russia_2017 = pd.read_csv('2017\premier-liga_2017.csv')
df_portugal_2017 = pd.read_csv('2017\primeira-liga_2017.csv')
df_scotland_2017 = pd.read_csv('2017\scottish-premiership_2017.csv')


# List of all DataFrames
dfs = [
    df_holland, df_belgium, df_russia, df_portugal, df_scotland,
    df_holland_2020, df_belgium_2020, df_russia_2020, df_portugal_2020, df_scotland_2020,
    df_holland_2019, df_belgium_2019, df_russia_2019, df_portugal_2019, df_scotland_2019,
    df_holland_2018, df_belgium_2018, df_russia_2018, df_portugal_2018, df_scotland_2018,
    df_holland_2017, df_belgium_2017, df_russia_2017, df_portugal_2017, df_scotland_2017
]

df_2021 = pd.concat([df_holland, df_belgium, df_russia, df_portugal, df_scotland],ignore_index=True)
df_2020 = pd.concat([df_holland_2020, df_belgium_2020, df_russia_2020, df_portugal_2020, df_scotland_2020],ignore_index=True)
df_2019 = pd.concat([df_holland_2019, df_belgium_2019, df_russia_2019, df_portugal_2019, df_scotland_2019],ignore_index=True)
df_2018 = pd.concat([df_holland_2018, df_belgium_2018, df_russia_2018, df_portugal_2018, df_scotland_2018],ignore_index=True)
df_2017 = pd.concat([df_holland_2017, df_belgium_2017, df_russia_2017, df_portugal_2017, df_scotland_2017],ignore_index=True)

df_transfers_top5 = pd.read_csv('players_transfers__top5_leagues.csv')
# Concatenate all DataFrames
df = pd.concat(dfs, ignore_index=True)
df.head()
df_transfers_out_of_top_5 = [df_2021, df_2020, df_2019, df_2018, df_2017]

In [694]:
num_transfers = 0
for i in range (len(df_transfers_out_of_top_5)):
  print ("before", len(df_transfers_out_of_top_5[i]))
  df_transfers_out_of_top_5[i] = preprocess_transfers(df_transfers_out_of_top_5[i])
  num_transfers += len(df_transfers_out_of_top_5[i])
  print ("after", len(df_transfers_out_of_top_5[i]))

print ("total transfers", num_transfers)



before 1772
Duplicate records found: 0
after 290
before 2609
Duplicate records found: 0
after 381
before 2635
Duplicate records found: 0
after 475
before 2631
Duplicate records found: 0
after 452
before 2779
Duplicate records found: 0
after 469
total transfers 2067


In [695]:
full_joined_df_out_of_top_5 = pd.DataFrame(columns = df_24.columns.tolist())
for i in range (len(df_transfers_out_of_top_5)):
    df_fifa_this_year = df_24[df_24['fifa_version'] ==(24 - i)]
    fifa_with_transfer = pd.merge(df_fifa_this_year, df_transfers_out_of_top_5[i], on='name', how='inner')
    fifa_with_transfer = fifa_with_transfer.drop_duplicates(subset=['name'])
    dfs_to_merge = [full_joined_df_out_of_top_5, fifa_with_transfer]
    full_joined_df_out_of_top_5 = pd.concat(dfs_to_merge, ignore_index=True)
    print("iteration", i, full_joined_df_out_of_top_5['fifa_version'].unique(), len(full_joined_df_out_of_top_5))

print((full_joined_df_out_of_top_5.columns.tolist()))
display(full_joined_df_out_of_top_5)
print(len(full_joined_df_out_of_top_5))
full_joined_df_out_of_top_5.to_csv('merged_data.csv', index=False)

full_joined_df_out_of_top_5.head()


iteration 0 [24.] 75
iteration 1 [24. 23.] 155
iteration 2 [24. 23. 22.] 258
iteration 3 [24. 23. 22. 21.] 377
iteration 4 [24. 23. 22. 21. 20.] 492
['fifa_version', 'player_positions', 'overall', 'potential', 'height_cm', 'weight_kg', 'club_name', 'club_position', 'club_contract_valid_until_year', 'nationality_name', 'nation_position', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions'

C:\Users\Yuval Margolin\AppData\Local\Temp\ipykernel_38856\556064823.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_joined_df_out_of_top_5 = pd.concat(dfs_to_merge, ignore_index=True)


,fifa_version,player_positions,overall,potential,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,nationality_name,...,lcb,cb,rcb,rb,gk,name,age,fee,position,league
0,24.00,"RM, ST",82,84,178,77,Borussia Dortmund,RM,2026.00,Netherlands,...,51+2,51+2,51+2,59+2,18+2,Donyell Malen,22.00,30000000.00,Centre-Forward,Eredivisie
1,24.00,"CAM, CM, RW",81,81,182,75,Ajax,RW,2025.00,Netherlands,...,51+3,51+3,51+3,59+3,18+3,Steven Berghuis,29.00,5500000.00,Right Winger,Eredivisie
2,24.00,"CM, CDM",80,83,183,77,Atalanta,CAM,2025.00,Netherlands,...,77+2,77+2,77+2,77+2,19+2,Teun Koopmeiners,23.00,12000000.00,Defensive Midfield,Eredivisie
3,24.00,"CM, LM, RB",79,80,179,71,Benfica,SUB,2027.00,Norway,...,76+2,76+2,76+2,77+2,20+2,Fredrik Aursnes,25.00,450000.00,Defensive Midfield,Eredivisie
4,24.00,CB,79,79,183,82,Feyenoord,RCB,2026.00,Austria,...,79,79,79,74+2,15+2,Gernot Trauner,29.00,1000000.00,Centre-Back,Eredivisie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,20.00,"ST, CAM",66,67,178,67,Aberdeen,SUB,2021.00,Scotland,...,43+2,43+2,43+2,48+2,15+2,Stevie May,24.00,450000.00,Centre-Forward,Scottish Premiership
488,20.00,CM,65,71,190,84,Hibernian,SUB,2020.00,Lithuania,...,60+2,60+2,60+2,59+2,9+2,Vykintas Slivka,22.00,100000.00,Central Midfield,Scottish Premiership
489,20.00,CB,65,69,195,78,AS Eupen,RCB,2022.00,Netherlands,...,64+2,64+2,64+2,57+2,15+2,Menno Koch,23.00,100000.00,Centre-Back,Eredivisie
490,20.00,CB,65,69,191,83,Livorno,LCB,2024.00,Slovenia,...,64+2,64+2,64+2,57+2,18+2,Matija Boben,23.00,300000.00,Centre-Back,Premier Liga


492


,fifa_version,player_positions,overall,potential,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,nationality_name,...,lcb,cb,rcb,rb,gk,name,age,fee,position,league
0,24.00,"RM, ST",82,84,178,77,Borussia Dortmund,RM,2026.00,Netherlands,...,51+2,51+2,51+2,59+2,18+2,Donyell Malen,22.00,30000000.00,Centre-Forward,Eredivisie
1,24.00,"CAM, CM, RW",81,81,182,75,Ajax,RW,2025.00,Netherlands,...,51+3,51+3,51+3,59+3,18+3,Steven Berghuis,29.00,5500000.00,Right Winger,Eredivisie
2,24.00,"CM, CDM",80,83,183,77,Atalanta,CAM,2025.00,Netherlands,...,77+2,77+2,77+2,77+2,19+2,Teun Koopmeiners,23.00,12000000.00,Defensive Midfield,Eredivisie
3,24.00,"CM, LM, RB",79,80,179,71,Benfica,SUB,2027.00,Norway,...,76+2,76+2,76+2,77+2,20+2,Fredrik Aursnes,25.00,450000.00,Defensive Midfield,Eredivisie
4,24.00,CB,79,79,183,82,Feyenoord,RCB,2026.00,Austria,...,79,79,79,74+2,15+2,Gernot Trauner,29.00,1000000.00,Centre-Back,Eredivisie
